In [1]:
import joblib
import pandas as pd
import numpy as np
import sparse
import json

### Vitals

In [2]:
df_time_series = joblib.load('out_vitals/df_time_series.joblib')
discretization_bins = json.load(open('/data/covid/process_data/FIDDLE_2018-2019/out_vitals/discretization.json', 'r'))
feature_names = json.load(open('/data/covid/process_data/FIDDLE_2018-2019/out_vitals/X_all.feature_names.json', 'r'))
len(discretization_bins)

30

In [3]:
cols_useful = list(discretization_bins.keys())

In [4]:
df_time_series = df_time_series[cols_useful]

In [ ]:
df_time_series

In [6]:
from steps_2 import *
from helpers import *

In [7]:
df = df_time_series

In [8]:
pool = multiprocessing.Pool(multiprocessing.cpu_count())
out_0 = list(tqdm(pool.imap_unordered(
    smart_qcut_bins,
    [(df[col], discretization_bins[col]) for col in df.columns]), total=len(df.columns)
))
cols_data, dis_bins = zip(*out_0)
out = list(tqdm(pool.imap_unordered(dummify, cols_data), total=len(df.columns)
))
pool.close()
pool.join()

100%|██████████| 30/30 [00:12<00:00,  2.35it/s]


In [9]:
df_features = pd.concat(out, axis=1).sort_index(axis=1)

In [10]:
# Drop any values='missing' for now
df_features = df_features.loc[:, [col for col in df_features.columns if 'missing' not in col]]

In [11]:
# Make sure same features
set(df_features.columns) <= set(feature_names)

True

In [12]:
df_features = df_features.reindex(columns=feature_names, fill_value=0)

In [ ]:
df_features

In [14]:
sdf = df_features.astype(pd.SparseDtype(int, fill_value=0))
X_all = sparse.COO(sdf.sparse.to_coo())

In [15]:
sparse.save_npz('out_vitals/X_all.npz', X_all)
df_time_series[[]].to_csv('out_vitals/X_all.IDs.csv')
# json.dump(list(X_all_feature_names), open('out_meds/X_all.feature_names.json', 'w'), sort_keys=True)
# json.dump(discretization_bins, open('out_meds/discretization.json', 'w'))